<a href="https://colab.research.google.com/github/BALU2000-creator/Sentement_Sentithon/blob/main/Sentithon_Twitter_DataExtract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install snscrape

In [ ]:
!pip install googletrans==3.1.0a0

In [ ]:
!pip install yake

In [ ]:
!!pip install langdetect

In [ ]:
!pip install clean-text

In [6]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
from tqdm import tqdm
from googletrans import Translator
from langdetect import detect
import re
from cleantext import clean
import yake

In [7]:
class keywords_extract():
  def get_key_words(self, text):
    kw_extractor = yake.KeywordExtractor()
    to_be_search = [text]                                      
    language = "en"
    max_ngram_size = 3
    deduplication_threshold = 0.9
    numOfKeywords = 20
    custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
    keywords = custom_kw_extractor.extract_keywords(text)
    for kw in keywords:
      to_be_search.append(kw[0])
    return to_be_search

In [8]:
translator = Translator()

def pre_processor(text):
  text = re.sub(r"\s*@\w*\s*", " ", text).strip()
  text = re.sub(r"http\S+|http[s]?://\S+|http://\S+|https://\S+", "", text)
  text = re.sub(r"[#\n\t!]",'', text)
  return text
text = "@desimojito !Yea#h Morbi bridge collapse, rampant corruption, 2nd wave COVID mismanagement development 🔔\n\n‘Bus Abdul ka screhttps://www.java2blog.comw tight karna hai’ is what they voted for. 🔔 atmanirbhar 😂🤣😂',"
pre_processor(text)

def lang_process(text):
  text = pre_processor(text)
  # print('lang_process- text', text)
  lan = detect(text)
  # print('lannnnn')
  if lan!='en':
    translated_obj = translator.translate(text, dest='en')  
    text = translated_obj.text
  return text, lan



In [15]:
class Process_Twitter():

  def __init__(self, topic='Atmanirbhar Bharat: Local Vocalist', selection = 'Public', since='2021-11-01', untill='2022-12-09', maximum_tweets=300):
    keywords_extractor = keywords_extract()
    self.to_be_search = keywords_extractor.get_key_words(topic)
    self.len_of_df = maximum_tweets
    print(self.to_be_search)
    self.since = since
    self.untill = untill
    self.total_langs = []

    self.top_journalists = [('anjana om kashyap','anjanaomkashyap'),   #Top journalists and Their Twitter username
                   ('abhisar sharma','abhisar_sharma'),
                   ('vikram chandra','vikramchandra'),
                   ('Sudhir Chaudhary','sudhirchaudhary'), 
                   ('swetha singh','zoo_bear'),
                   ('rajat sharma','RajatSharmaLive'), 
                   ('nidhi razdan','Nidhi'), 
                   ('Rajdeep Sardesai','sardesairajdeep'),
                   ('Annap Goswami','AnnapGoswami'), 
                   ('Ravish Kumar', 'ravishndtv'),
                   ('Narendra Modi', 'narendramodi')]
    self.reference_tweet_attributes = ['tweet.date', 'tweet.id', 'tweet.content', 'tweet.url',
                          'tweet.user.username', 'tweet.user.followersCount','tweet.replyCount',
                          'tweet.retweetCount', 'tweet.likeCount', 'tweet.quoteCount', 'tweet.lang',
                          'tweet.outlinks', 'tweet.media', 'tweet.retweetedTweet', 'tweet.quotedTweet',
                          'tweet.inReplyToTweetId', 'tweet.inReplyToUser', 'tweet.mentionedUsers',
                          'tweet.coordinates', 'tweet.place', 'tweet.hashtags', 'tweet.cashtags']
    self.of_public_df  = None
    self.of_journals_df= None
    if selection=='Both':
      self.of_public_df = self.create_public()
      self.of_journals_df = self.create_journalists()
    elif selection == 'Journalists':
      self.of_public_df = self.create_journalists()
    else:
      self.of_public_df = self.create_public()

   
  def create_public(self):
    attributes_container = []
    # Using TwitterSearchScraper to scrape data and append tweets to list
    for search in tqdm(self.to_be_search):
      print('search.....', search)
      for i,tweet in enumerate(sntwitter.TwitterSearchScraper(f'{search} since:2021-11-01 until:2022-12-09').get_items()):
          text = tweet.content
          try:
            text = pre_processor(text)
            lan = detect(text)
            if lan!='en':
              translated_obj = translator.translate(text, dest='en')  
              text = translated_obj.text
          except:
            lan=None
            continue
          self.total_langs.append(lan)
          tweet.content = text
          if len(attributes_container)>self.len_of_df:
              break
          attributes_container.append([tweet.user.username, tweet.date, tweet.likeCount, tweet.content, tweet.url])
      if len(attributes_container)>self.len_of_df:
        break
    tweets_df = pd.DataFrame(attributes_container, columns=["User", "Date Created", "Number of Likes", "Tweet", "url"])
    print('create_public is succesful')
    del attributes_container
    return tweets_df
  
  def create_journalists(self):
    attributes_container_ = []
    top_journalists_accounts = []                
    for _,account in self.top_journalists:
      top_journalists_accounts.append(account)
    users_names = top_journalists_accounts
    # for n, k in tqdm(enumerate(users_name)):
    for user_name in tqdm(users_names):
      for i,tweet in enumerate(sntwitter.TwitterSearchScraper('from:{}'.format(user_name)).get_items()):
        if len(attributes_container_)>self.len_of_df:
          break
        text = tweet.content
        try:
          text = pre_processor(text)
          lan = detect(text)
          if lan!='en':
            translated_obj = translator.translate(text, dest='en')  
            text = translated_obj.text
        except:
          lan=None
          continue
        self.total_langs.append(lan)
        tweet.content = text

        # for keyword in to_be_search:
        #   if keyword in tweet.content:
        attributes_container_.append([tweet.user.username, tweet.date, tweet.likeCount, tweet.content, tweet.url])
      if len(attributes_container_)>self.len_of_df:
        break
    tweets_df = pd.DataFrame(attributes_container_, columns=["User", "Date Created", "Number of Likes", "Tweet", "url"])
    print('create_journalists is succesful')
    del attributes_container_
    return tweets_df

  def get_public_journalists_both(self):
    # if self.of_public_df and self.of_journals_df:
    total_df = pd.concat([self.of_public_df, self.of_journals_df], axis=0)
    return total_df, self.of_public_df, self.of_journals_df
    # else:
    #   print('some missing in Process_Twitter get_public_journalistsmethod')
    
  def get_total_languages(self):
    return self.total_langs
  
  def __del__(self):
    del total_df
    del self.of_public_df
    del self.of_journals_df
    print('Destructor called, Employee deleted.')

In [1]:
from numpy import maximum
search_key_words = ['Atmanirbhar Bharat: Local Vocalist',
'Digital India',
'Covid-19 Booster Vaccination Shots',
'recent Indian government initiative']
total_langs = []
for topic in search_key_words:
  x = Process_Twitter(topic = topic, selection='Both', maximum_tweets=300)
  ALl_dfs = x.get_public_journalists_both()
  total_langs.append(x.get_total_languages())
  del x
  file_names = ('TweetTotal_', 'TweetPublic_', 'TweetJournalists_')
  for i in range(3):
    df = list(ALl_dfs[i]['Tweet'])
    f = open(file_names[i]+topic+'.txt', 'w')
    for text in df:
      f.write(text)
      f.write('\n')